# Advanced Topics : Delegates

____

1. Generics
2. Delegates
3. Lamda Expressions
4. Events
5. Extension Methods
6. LINQ
7. Nulllable Types
8. Dynamic
9. Exception Handling
10. Asynchronous Programming



In [35]:
using System;
// using System.Collections.Generic;
// using System.Linq; 
// using System.Text;
// using System.Threading.Tasks;


## Delegates

____

It is an object that knows how to call a method, and normall used to call a GROUP of method. Also said to be a reference or pointer to a function

How we going to demostrate it

1. Firstly how we going to implement it, our way (or calle CUSTOM delegate)
2. Finally how .NET implementts it.


But why dont we call methods directly? why do we need to have this intermediate step? This technique allows us to create apps that are EXTENSIBLE and FLEXIBLE, and is often used when you design frameworks.

So we have the following code

1. Photo class, that represents an image
2. PhotoProcessor class, it has a Process method, which takes a path to file, loads the photo from this location, apply a set of filters (through a PhotoFilter class), and saves it 
3. PhotoFilter class, and has the ApplyFilter, ApplyContrast, Resize methods

Here we are designing a framework for processing photos...

But this code has a problem

1. It is not extensible?? What does that mean...

We have 3 filters, and another dev wants to add another filter that is not of the photofilter class. So he will need to add filter, and then compile the class and dependent classes again

But this dev that not have your souce code....this is problem. You created a class that is not extensible

With delegates we can make this framework extensible.

We can also use INTERFACES (using polymorphism) to solve this problem, but lets stick to delegates here (we've done interfaces before)

Lets run code...

In [36]:
public class Photo{
    
    public static Photo Load(string Path){
        return new Photo();
    }
    
    public void Save(){
        Console.WriteLine("Photo Saved");
    }
}

In [37]:
public class PhotoFilter{

    public void ApplyBrightness(Photo photo){
        Console.WriteLine("Apply Brightness");
    }
    
    public void ApplyContrast(Photo photo){
        Console.WriteLine("Apply Contrast");
    }
    
    public void Resize(Photo photo){
        Console.WriteLine("Apply Resize");
    }
}

In [38]:
public class PhotoProcessor{
    
    public void Process(string path){
        
        var photo = Photo.Load(path);
        
        var filters = new PhotoFilter();
        
        filters.ApplyBrightness(photo);
        filters.ApplyContrast(photo);
        filters.Resize(photo);
        
        photo.Save();
        
    }
        
}

In [39]:
var photoProcessor = new PhotoProcessor();
photoProcessor.Process("\\");
                       


Apply Brightness
Apply Contrast
Apply Resize
Photo Saved


## Lets add  delegate to make it extensible

(Remember we doing it our way first, later the .NET way)
First we beed to define a delegate type. Word delegate and then signature. The signature is what the delegate will be calling. So instance of the delegate will be to method of group of methods that the delegate will be calling


```
public delegate void PhotoFilterHandler(Photo photo)
```
Syntax

1. Access modifier
2. Word delegate
3. Then the signature of the delegate, In the form of the method we going to call. 
4. In the form of the method
    ```void XX(YY)```
5. That will be the name of the delegate we will be using.
6. (the delegate is defined like function, so we call it like function) - see main.

So we delegate a method that we want to be extensible, that is photofilter. So instead of hard coding the filters, we define (CONCRETE) where we calling it

And we pass a delegate to the Process method, that will do the processing, and we dont instantiate the PhotoFilter there anymore, we pass the delegate. And we just pass the photo object.  

```
 public void Process(string path,PhotoFilterHandler filterHandler){
      filterHandler(photo);
 }
```

So we can see that a delegate (our implemenation), is a ```type of``` in a sense.

Here is magic: The PhotoProcessor class, and Process method does not know what filter will be applied. The method/class making a call to the Process method, will supply (CONCRETE) the filter. That is responsibility of the client

No recompiling of clases.

And who is the client of this class, that will be main... here we need to supply the filter.


Here we going to create an instance of that delegate. Where is that delegate defined? In PhotoProcessor class...

We get get instance of PhotoFilters. And we call the delegate as follows:


The syntax for calling:


```
var filters = new PhotoFilter();
PhotoProcessor.PhotoFilterHandler filterHandler = filters.ApplyBrightness


```

1. We calling like class
2. And we apply assigment ```+=``` operator to add more methods.

And we pass the filterHandler to the process method

And we get the following:

```
PhotoProcessor.PhotoFilterHandler filterHandler = filters.ApplyBrightness;

filterHandler += filters.ApplyContrast;
filterHandler += filters.Resize;

```

So now, if another developer, client of the class, want to add a filter, he can do so, without any issues.

Lets add another filers, RemoveRedEye. But, it must confirm with the sigature of the method that the delegate is implementing.

```
void name (Photo photo)
```

That is a delegate is pointer to method that has this specific signature.


So the following is not touched

1. PhotoProcessor class
2. PhotoFilter class





In [40]:
public class Photo{
    
    public static Photo Load(string Path){
        return new Photo(); 
    }
    
    public void Save(){
        Console.WriteLine("Photo Saved");
    }
}

In [41]:
public class PhotoFilter{

    public void ApplyBrightness(Photo photo){
        Console.WriteLine("Apply Brightness");
    }
    
    public void ApplyContrast(Photo photo){
        Console.WriteLine("Apply Contrast");
    }
    
    public void Resize(Photo photo){
        Console.WriteLine("Apply Resize");
    }
}

In [42]:
public class PhotoProcessor{
    
    public delegate void PhotoFilterHandler(Photo photo);
        
    
    public void Process(string path,PhotoFilterHandler filterHandler){
        
        var photo = Photo.Load(path);
        
        filterHandler(photo);
        
        photo.Save();
        
    }
        
}

In [43]:
var processor = new PhotoProcessor();
                       
var filters = new PhotoFilter();
PhotoProcessor.PhotoFilterHandler filterHandler = filters.ApplyBrightness;

filterHandler += filters.ApplyContrast;
filterHandler += filters.Resize;

processor.Process("photo.jpg",filterHandler);
 

    



Apply Brightness
Apply Contrast
Apply Resize
Photo Saved


In [44]:
public static void RemoveRedEye(Photo photo){
        Console.WriteLine("Apply Remove Red Eye");
}

In [45]:
filterHandler += RemoveRedEye;
processor.Process("photo.jpg",filterHandler);

Apply Brightness
Apply Contrast
Apply Resize
Apply Remove Red Eye
Photo Saved


## What happens under the hood

Derives from System.MulticlassDelegate class, derived from System.Delegate, which has the following public properties:


1. Method, represent a method that the target is pointing to
2. Target, represents the location or class where that method is located


In our cases: 

```
method = ApplyBrightness (method) (what we abstracting)
target = PhotoFilters (class)
```


## Now lets use the Built-in .NET Delegate 

_____

So instead of defining a custom delegate, as we did above, lets use .net built-in delegate class to perform the above

In .NET we have two generic delegates

1. Action (two version generic and non generic). It has up to 16 overloads.
2. Func


 
```Action``` can point to method that can take up to 16 parameters, and that returns void

```Func``` can point to method that can take up to 16+ parameters, and but it returns a value

So this gives you a lot of flexibility, that means there is no need to create your own custom delegates.

Lets remove the custom PhotoFilterHandler, and use the generic Action delegate


So in PhotoProcessor class (Process method), we say you can pass any delegate that takes a Photo as an argument, and returns void, and we define this Action delegate as filterHandler.


```
public void Process(string path,Action<Photo> filterHandler){

``` 

Lets update main, to use Action, instead of the custom delegate


Replace

```
PhotoProcessor.PhotoFilterHandler filterHandler = filters.ApplyBrightness;

```

with 

```
Action<Photo> filterHandler = filters.ApplyBrightness;

```



In [46]:
public class Photo{
    
    public static Photo Load(string Path){
        return new Photo(); 
    }
    
    public void Save(){
        Console.WriteLine("Photo Saved");
    }
}

In [47]:
public class PhotoFilter{

    public void ApplyBrightness(Photo photo){
        Console.WriteLine("Apply Brightness");
    }
    
    public void ApplyContrast(Photo photo){
        Console.WriteLine("Apply Contrast");
    }
    
    public void Resize(Photo photo){
        Console.WriteLine("Apply Resize");
    }
}

In [48]:
public class PhotoProcessor{
    
    public void Process(string path,Action<Photo> filterHandler){
        
        var photo = Photo.Load(path);
        
        filterHandler(photo);
        
        photo.Save();
        
    }
        
}

In [49]:
var processor = new PhotoProcessor();
                       
var filters = new PhotoFilter();

Action<Photo> filterHandler = filters.ApplyBrightness;

filterHandler += filters.ApplyContrast;
filterHandler += filters.Resize;

processor.Process("photo.jpg",filterHandler);
 

    

Apply Brightness
Apply Contrast
Apply Resize
Photo Saved


## Summary

We setting up a pointer to a function, by using a delegate

Essentially is object that KNOWS how to call a method or group of methods.

And we use if extensibility and flexibility

But we can use INTERFACES to achieve the above as will, but how do we decide?


The MSDN guideline suggests that we use delegates when we have some eventing design pattern or caller does not need to access other properties, methods or interfaces of the object implementing the method. 
 
